# Imports
## Keeping your API Key hidden

1. Create a new file and name it api_key.py.
2. Add the API key to the file: API_KEY = "your_api_key_here"
3. Save the file in the same directory where your main Python script is located.

## Please look at [Configuration](Configuration) section below first before modifying code.
- Task: Select the task
- Eval: Evaluation type (soft or hard)
- Prompt: If you would like to change the prompt from a text file instead of in the code.
- API Key: Your imported API Key
- Data Frame: Data Frame created to pass through in the section

In [13]:
import pandas as pd
import json
import openai
import re
import time
import math
import aiohttp
import asyncio
import logging
import os
from api_key import API_KEY
import nest_asyncio
nest_asyncio.apply()

# Batch Functions

In [2]:
def json_df(json_location, dataframe_name):
    """
    Load a JSON file and convert it into a pandas DataFrame.

    Parameters:
    json_location (str): The file path of the JSON file.
    dataframe_name (str): The name of the resulting DataFrame.

    Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
    """
    with open(json_location, 'r') as f:
        data = json.load(f)
    dataframe_name = pd.DataFrame.from_dict(data, orient='index')
    return dataframe_name

def load_prompt(file_path):
    """
    Load the prompt from a file.

    Args:
        file_path (str): The path to the file containing the prompt.

    Returns:
        str: The loaded prompt.

    """
    with open(file_path, 'r') as file:
        prompt = file.readline().strip()
    return prompt

def save_responses(responses_dict, output_dir='.', base_filename='NO_FILE_NAME_RMIT', increment_filename=True):
    """
    Save responses to a JSON file.

    Args:
        responses_dict (dict): A dictionary containing the responses to be saved.
        output_dir (str, optional): The directory where the JSON file will be saved. Defaults to '.'.
        base_filename (str, optional): The base filename for the JSON file. Defaults to 'NO_FILE_NAME_RMIT'.
        increment_filename (bool, optional): Whether to increment the filename if it already exists. Defaults to True.

    Raises:
        ValueError: If responses_dict is not a dictionary.

    """
    if not isinstance(responses_dict, dict):
        raise ValueError("responses_dict must be a dictionary.")
    
    os.makedirs(output_dir, exist_ok=True)
    filename = f"{base_filename}.json" if not increment_filename else get_next_filename(output_dir, base_filename)
    full_path = os.path.join(output_dir, filename)
    
    # Convert dictionary to list for JSON dumping
    responses_list = list(responses_dict.values())

    try:
        with open(full_path, 'w') as file:
            json.dump(responses_list, file, indent=4)
        print(f"Data saved to {filename}")
    except IOError as e:
        print(f"Failed to save data: {e}")

def get_next_filename(output_dir, base_filename):
    """
    Get the next available filename for a given base filename in the specified output directory.

    Args:
        output_dir (str): The directory where the files are stored.
        base_filename (str): The base filename to be used.

    Returns:
        str: The next available filename in the format "{base_filename}_{number}.json".

    """
    pattern = re.compile(rf"{re.escape(base_filename)}_(\d+).json")
    max_number = 0
    for filename in os.listdir(output_dir):
        match = pattern.match(filename)
        if match:
            number = int(match.group(1))
            if number > max_number:
                max_number = number
    return f"{base_filename}_{max_number + 1}.json"


# Async Functions

In [10]:
async def fetch_response(session, tweet, api_key, eval, prompt, df, idx, task):
    """
    Fetches a response from the OpenAI API based on the given parameters.

    Args:
        session (aiohttp.ClientSession): The aiohttp client session.
        tweet (str): The tweet for which the response is requested.
        api_key (str): The API key for accessing the OpenAI API.
        eval (str): The evaluation mode ('hard' or 'soft').
        prompt (str): The prompt value for the conversation.
        df (pandas.DataFrame): The DataFrame containing study levels and gender information.
        idx (int): The index of the current row in the DataFrame.
        task (int): The task number.

    Returns:
        dict: A dictionary containing the tweet and the response from the API.

    Raises:
        Exception: If an error occurs during the API request.

    """
    api_key = API_KEY
    delimiter = "####"
    response_value = ""
    prompt_value = f""""sexism, prejudice or discrimination based on sex or gender, especially against women and girls. Although its origin is unclear, the term sexism emerged from the “second-wave” feminism of the 1960s through ’80s and was most likely modeled on the civil rights movement’s term racism (prejudice or discrimination based on race). Sexism can be a belief that one sex is superior to or more valuable than another sex. It imposes limits on what men and boys can and should do and what women and girls can and should do. The concept of sexism was originally formulated to raise consciousness about the oppression of girls and women, although by the early 21st century it had sometimes been expanded to include the oppression of any sex, including men and boys, intersex people, and transgender people. You are a robot who detects sexism from text given in the prompt."""    
    column_value = f"""For each response, consider the perspective of individuals representing the following study levels: {df.study_levels_annotators[idx]} and gender: {df.gender_annotators[idx]}."""
    if eval == 'hard':
        if task == 1:
            response_value = f"""Give me 1 answer with [NO] or [YES]"""
        elif task == 2:
            response_value = f"""Give me 1 answer: [NO], [DIRECT], [REPORTED] or [JUDGEMENTAL]"""
        elif task == 3:
            response_value = f"""Give me 1 to 5 answers using commas for each answer. If it is sexist, classify it. Do not say YES. This is a multi-label task, so that more than one of the following labels may be assigned to each tweet: [NO], [IDEOLOGICAL-INEQUALITY], [STEREOTYPING-DOMINANCE], [OBJECTIFICATION], [SEXUAL-VIOLENCE], or [MISOGYNY-NON-SEXUAL-VIOLENCE]"""
            
    if eval == 'soft':
        if task == 1:
            response_value = f"""Give me 6 answers with NO or YES. Format: [NO], [YES]"""
        elif task == 2:
            response_value = f"""Give me 6 answers with NO, DIRECT, REPORTED or JUDGEMENTAL using commas for each answer. Example: [NO], [DIRECT], [REPORTED], [JUDGEMENTAL], [JUDGEMENTAL], [NO]"""
        elif task == 3:
            response_value = f"""Give me 6 answers with NO, IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINANCE, OBJECTIFICATION, SEXUAL-VIOLENCE, or MISOGYNY-NON-SEXUAL-VIOLENCE using commas for each answer. Example: [NO], [IDEOLOGICAL-INEQUALITY], [STEREOTYPING-DOMINANCE], [OBJECTIFICATION], [SEXUAL-VIOLENCE], [MISOGYNY-NON-SEXUAL-VIOLENCE]"""
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {"role": "system", "content": f""""{prompt_value} {column_value} {response_value}"""},
            # {"role": "system", "content": f""""{prompt_value} {response_value}"""},
            {"role": "user", "content": delimiter + " " + tweet + delimiter + " "}
        ]
    }
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    try:
        while True:
            async with session.post('https://api.openai.com/v1/chat/completions', json=payload, headers=headers) as response:
                if response.status == 429:
                    retry_after = float(response.headers.get('Retry-After', 0.12))
                    await asyncio.sleep(retry_after)
                    continue
                if response.status != 200:
                    error_message = await response.text()
                    print(f"HTTP error {response.status}: {error_message}")
                    return {"tweet": tweet, "response": f"HTTP error {response.status}: {error_message}"}
                result = await response.json()
                if 'choices' in result and result['choices']:
                    return {"tweet": tweet, "response": result['choices'][0]['message']['content']}
                    print('49')
                else:
                    print("Unexpected response structure:", json.dumps(result, indent=4))
                    return {"tweet": tweet, "response": "Error: Unexpected API response"}
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return {"tweet": tweet, "response": f"Error: {str(e)}"}
        


def normalize_response(response, eval, task):
    """
    Normalize the response based on the evaluation method and task.

    Args:
        response (str): The response to be normalized.
        eval (str): The evaluation method ('soft' or 'hard').
        task (int): The task number (1, 2, or 3).

    Returns:
        dict or list: If eval is 'soft', returns a dictionary with normalized counts of response categories.
                      If eval is 'hard', returns a list of normalized response categories.

    """
    answers = response.replace('[', '').replace(']', '').split(',')
    answers = [answer.strip().upper() for answer in answers]  # Clean and normalize case
    # Define response categories based on the task and evaluation method
    if eval == 'soft':
        if task == 1:
            counts = {'NO': 0, 'YES': 0}
        elif task == 2:
            counts = {'NO': 0, 'DIRECT': 0, 'REPORTED': 0, 'JUDGEMENTAL': 0}
        elif task == 3:
            counts = {'NO': 0, 'IDEOLOGICAL-INEQUALITY': 0, 'STEREOTYPING-DOMINANCE': 0, 'OBJECTIFICATION': 0,'SEXUAL-VIOLENCE':0, 'MISOGYNY-NON-SEXUAL-VIOLENCE':0}
        else:
            # Return empty dictionary for unspecified tasks or evaluation modes
            print(answers)
            return {}

        # Count responses
        for answer in answers:
            if answer in counts:
                counts[answer] += 1

        # Normalize counts by the total number of responses
        total = sum(counts.values())
        if total > 0:
            for key in counts:
                counts[key] = counts[key] / total
        return counts
    else:
        return answers

async def main(**kwargs):
    """
    Fetches and processes tweets asynchronously.

    Args:
        **kwargs: Keyword arguments containing the following parameters:
            - dataframe: The dataframe containing the tweets.
            - api_key: The API key for accessing the API.
            - prompt: The prompt for generating responses.
            - eval: The evaluation mode for the response.
            - task: The task number for processing the tweets.

    Returns:
        A tuple containing two dictionaries:
        - responses_dict: A dictionary mapping row IDs to normalized response values.
        - raw_responses: A dictionary mapping row IDs to raw response values.
    """
    tweets = kwargs.get('dataframe')['tweet'].to_list()
    api_key = kwargs.get('api_key')
    prompt = kwargs.get('prompt')
    eval = kwargs.get('eval')
    df = kwargs.get('dataframe')
    task = kwargs.get('task')
    
    async with aiohttp.ClientSession() as session:
        responses_dict = {}
        raw_responses = {}  # Dictionary to store non-normalized responses
        batch_size = 500  # Define the number of requests after which to pause
        requests_per_second = batch_size / 60  # Calculate the delay needed between each request to adhere to rate limits
        for i, tweet in enumerate(tweets):
            # Pause execution to respect rate limits after every batch of 500 requests
            if i % batch_size == 0 and i != 0:
                print(f"Processed {i} tweets, sleeping for 60 seconds to respect rate limits...")
                await asyncio.sleep(60)

            # Fetch and process each tweet
            if task == 1:
                response = await fetch_response(session, tweet, api_key, eval, prompt, df, i, task)
                if 'response' in response:
                    normalized_values = normalize_response(response['response'], eval, task)
                    row_id = df[df['tweet'] == tweet]['id_EXIST'].values[0]
                    responses_dict[row_id] = {
                        'id': row_id,
                        'value': normalized_values,
                        'test_case': "EXIST2024"
                    }
                    # Store raw response
                    raw_responses[row_id] = {
                        'id': row_id,
                        'response': response['response'],
                        'test_case': "EXIST2024"
                    }
                else:
                    print(f"Error or unexpected format in response for tweet ID {tweet}: {response}")
            if task == 2:
                response = await fetch_response(session, tweet, api_key, eval, prompt, df, i, task)
                if 'response' in response:
                    normalized_values = normalize_response(response['response'], eval, task)
                    row_id = df[df['tweet'] == tweet]['id_EXIST'].values[0]
                    responses_dict[row_id] = {
                        'id': row_id,
                        'value': normalized_values,
                        'test_case': "EXIST2024"
                    }
            if task == 3:
                response = await fetch_response(session, tweet, api_key, eval, prompt, df, i, task)
                if 'response' in response:
                    normalized_values = normalize_response(response['response'], eval, task)
                    row_id = df[df['tweet'] == tweet]['id_EXIST'].values[0]
                    responses_dict[row_id] = {
                        'id': row_id,
                        'value': normalized_values,
                        'test_case': "EXIST2024"
                    }
                    
                # Sleep to spread requests evenly across the rate limit period
                await asyncio.sleep(1 / requests_per_second)

        return responses_dict, raw_responses

# Configuration

In [7]:
# json_location = 'dev/EXIST2024_dev.json' # Test model
json_location = 'test/EXIST2023_test_clean.json' # For Official Submission
test_case = "EXIST2024"

df = json_df(json_location, 'df')
df = df.head(2)
"""*****After running task 3, make sure to change the prompt to fit set 2*****"""
test_params = {
    'task': 3, # 1 or 2 and then 3!?
    'eval': 'hard', # soft or hard
    'prompt': 'Prompts/prompt.txt', # If you would like to change the prompt from a text file instead of in the code.
    'api_key': API_KEY,
    'dataframe': df
    }
base_filename = f"task{test_params['task']}_{test_params['eval']}_RMITIR"

In [ ]:
if __name__ == "__main__":
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    responses, raw_responses = loop.run_until_complete(main(**test_params))

    # Saving the normalized responses if they exist
    if responses:
        save_responses(responses, output_dir='test_formats/gpt4/test', base_filename=base_filename, increment_filename=True)

    # Optionally save the raw responses too
    if raw_responses:
        save_responses(raw_responses, output_dir='raw_formats/gpt4/test', base_filename=base_filename, increment_filename=True)

# ID Match Check

In [ ]:
import json

# Function to load JSON data from a file
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Function to find mismatched values based on ID
def compare_values(data1, data2):
    mismatch = []  # List to collect IDs where values do not match
    # Create a dictionary from the second dataset for quick lookup
    data2_dict = {item['id']: item['value'] for item in data2}
    # Iterate through the first dataset and compare
    for item in data1:
        id = item['id']
        # Check if the ID exists in both datasets and the values do not match
        if id in data2_dict and item['value'] != data2_dict[id]:
            mismatch.append(id)  # Add to mismatch list
    return mismatch

# Load data from files (replace 'file1.json' and 'file2.json' with your actual file paths)
data1 = load_json('file1')
data2 = load_json('file2')
# Compare the files and get IDs with the same value
matching_ids = compare_values(data1, data2)

# Output the results
print("IDs Mismatched:", matching_ids)
print(len(matching_ids))